# Importing and Cleaning of Data 
From CSV file back to a "cleaned" CSV

In [ ]:
#Importing Libraries
import pandas as pd
from pathlib import Path 
import requests
import json
import time
import numpy as np
import re
from geopy.geocoders import Nominatim
from tqdm import tqdm

In [ ]:
#Read CSV file into a Dataframe
vaccine_providers = Path("../Resources/Vaccines.gov__COVID-19_vaccinating_provider_locations_20241118.csv")
providers_df = pd.read_csv(vaccine_providers)
providers_df.head()

In [ ]:
#Rename and reorganize columns of the DataFrame
renamed_df = providers_df.rename(columns={"loc_phone":"Phone Number", 
                                         "loc_name":"Location Name", 
                                         "loc_admin_street1":"Street Address",
                                         "loc_admin_city":"City", 
                                         "loc_admin_state":"State", 
                                         "loc_admin_zip":"Zip Code", 
                                         "web_address":"Website", 
                                         "insurance_accepted":"Insurance Accepted",
                                          "walkins_accepted": "Walk-Ins Accepted",
                                          "med_name":"Types of Vaccine", 
                                          "in_stock":"In Stock", 
                                          "quantity_last_updated":"Last Updated", 
                                          "latitude": "Latitude", 
                                          "longitude":"Longitude"})
vaccine_providers_df = renamed_df[["Location Name", 
                                   "Phone Number", 
                                   "Street Address", 
                                   "City", 
                                   "State", 
                                   "Zip Code",
                                  "Website",
                                  "Insurance Accepted",
                                  "Walk-Ins Accepted", 
                                  "Types of Vaccine",
                                  "In Stock",
                                  "Last Updated", 
                                  "Latitude", 
                                  "Longitude"]]

In [ ]:
#Check data types
vaccine_providers_df.dtypes

In [ ]:
#Convert all phone numbers to same format
#Convert phone numbers with a decimal to dashes
def convert_phone_number(phone):
    pattern = r"(\d{3})(\d{3})(\d{4})\.0"
    replacement = r"\1-\2-\3"
    return re.sub(pattern, replacement, phone)

# Apply the function to the 'Phone Number' column
vaccine_providers_df['Phone Number'] = vaccine_providers_df['Phone Number'].apply(convert_phone_number)

#Convert phone numbers from having (Areacode in parentheses) to 314-123-4567 format.
def convert_phone_number(phone):
    pattern = r"\((\d{3})\) (\d{3})-(\d{4})"
    replacement = r"\1-\2-\3"
    return re.sub(pattern, replacement, phone)

# Apply the function to the 'Phone Number' column
vaccine_providers_df['Phone Number'] = vaccine_providers_df['Phone Number'].apply(convert_phone_number)

#Convert phone numbers from 3141234567 to 314-123-4567 format.
def convert_phone_number(phone):
    pattern = r"(\d{3})(\d{3})(\d{4})"
    replacement = r"\1-\2-\3"
    return re.sub(pattern, replacement, phone)

# Apply the function to the 'Phone Number' column
vaccine_providers_df['Phone Number'] = vaccine_providers_df['Phone Number'].apply(convert_phone_number)

vaccine_providers_df.head()

In [ ]:
#Clean all zip codes to be 5 digits only
#Fix the zipcode to have five digits
def fix_zipcode(zipcode):
    match = re.match(r'(\d{5})-\d{4}', zipcode)
    if match:
        return match.group(1)
vaccine_providers_df['Zip Code'] = vaccine_providers_df['Zip Code'].apply(fix_zipcode)
# Display the updated DataFrame
vaccine_providers_df

In [ ]:
#Convert the insuracne and walk-ins columns to booleans
def convert_Insurance_to_boolean(value):
    if value in ['True', True]:
        return True
    else:
        return False
vaccine_providers_df['Insurance Accepted'] = vaccine_providers_df['Insurance Accepted'].apply(convert_Insurance_to_boolean)

def convert_Walk_Ins_to_boolean(value):
    if value in ['True', True]:
        return True
    else:
        return False
vaccine_providers_df['Walk-Ins Accepted'] = vaccine_providers_df['Walk-Ins Accepted'].apply(convert_Walk_Ins_to_boolean)


In [ ]:
#Check data types for all columns
vaccine_providers_df.dtypes

In [ ]:
#Save DataFrame to a csv
vaccine_providers_df.to_csv("Vaccine Providers.csv")

In [ ]:
#Save DataFrame to a csv
vaccine_providers_df.to_json("providers.json")

In [ ]:

# Check for NaN values
nan_count = vaccine_providers_df.isnull().sum()

# Display the count of NaN values for each column
print("Count of NaN values in each column:")
print(nan_count)

# To get the locations of NaN values
nan_locations = vaccine_providers_df.isnull()
print("\nLocations of NaN values:")
print(nan_locations)

In [13]:
zipcode_drop_df = vaccine_providers_df.drop(columns=['Zip Code'])
print(zipcode_drop_df)

                    Location Name  Phone Number                Street Address  \
0        CVS Pharmacy, Inc. #5340  504-737-6242          9643-B JEFFERSON HWY   
1        CVS Pharmacy, Inc. #2113  215-836-1323         1600 WADSWORTH AVENUE   
2       CVS Pharmacy, Inc. #16504  619-315-0016               2911 JAMACHA RD   
3              Rite Aid #RA105255  253-851-6939     4818 POINT FOSDICK DR.,NW   
4            Walmart Inc #10-1447  715-392-9520                3705 Tower Ave   
...                           ...           ...                           ...   
135780      SAV-ON PHARMACY #3739  215-942-4894               105 E STREET RD   
135781         Rite Aid #RA101653  603-532-6955        14 PETERBOROUGH STREET   
135782   CVS Pharmacy, Inc. #4378  540-989-6633           4400 BRAMBLETON AVE   
135783         Rite Aid #RA101963  724-438-9799      262 CONNELLSVILLE STREET   
135784        VONS PHARMACY #1962  951-695-7273  29530 RANCHO CALIFORNIA ROAD   

                        Cit

In [14]:
nozipcode_latlng_dropna_df = zipcode_drop_df.dropna()

# Display the cleaned DataFrame
print("\nDataFrame after removing rows with null values:")
print(nozipcode_latlng_dropna_df)


DataFrame after removing rows with null values:
                    Location Name  Phone Number                Street Address  \
0        CVS Pharmacy, Inc. #5340  504-737-6242          9643-B JEFFERSON HWY   
1        CVS Pharmacy, Inc. #2113  215-836-1323         1600 WADSWORTH AVENUE   
2       CVS Pharmacy, Inc. #16504  619-315-0016               2911 JAMACHA RD   
3              Rite Aid #RA105255  253-851-6939     4818 POINT FOSDICK DR.,NW   
4            Walmart Inc #10-1447  715-392-9520                3705 Tower Ave   
...                           ...           ...                           ...   
135780      SAV-ON PHARMACY #3739  215-942-4894               105 E STREET RD   
135781         Rite Aid #RA101653  603-532-6955        14 PETERBOROUGH STREET   
135782   CVS Pharmacy, Inc. #4378  540-989-6633           4400 BRAMBLETON AVE   
135783         Rite Aid #RA101963  724-438-9799      262 CONNELLSVILLE STREET   
135784        VONS PHARMACY #1962  951-695-7273  29530 RANCH

In [15]:
#Save DataFrame to a csv
nozipcode_latlng_dropna_df.to_csv("nozipcode.csv")

In [16]:
dropvaccinetypes_df = nozipcode_latlng_dropna_df.drop(columns=['Types of Vaccine'])
print(dropvaccinetypes_df)

                    Location Name  Phone Number                Street Address  \
0        CVS Pharmacy, Inc. #5340  504-737-6242          9643-B JEFFERSON HWY   
1        CVS Pharmacy, Inc. #2113  215-836-1323         1600 WADSWORTH AVENUE   
2       CVS Pharmacy, Inc. #16504  619-315-0016               2911 JAMACHA RD   
3              Rite Aid #RA105255  253-851-6939     4818 POINT FOSDICK DR.,NW   
4            Walmart Inc #10-1447  715-392-9520                3705 Tower Ave   
...                           ...           ...                           ...   
135780      SAV-ON PHARMACY #3739  215-942-4894               105 E STREET RD   
135781         Rite Aid #RA101653  603-532-6955        14 PETERBOROUGH STREET   
135782   CVS Pharmacy, Inc. #4378  540-989-6633           4400 BRAMBLETON AVE   
135783         Rite Aid #RA101963  724-438-9799      262 CONNELLSVILLE STREET   
135784        VONS PHARMACY #1962  951-695-7273  29530 RANCHO CALIFORNIA ROAD   

                        Cit

In [17]:
dropvaccinetypes_df.head()

,Location Name,Phone Number,Street Address,City,State,Website,Insurance Accepted,Walk-Ins Accepted,In Stock,Last Updated,Latitude,Longitude
0,"CVS Pharmacy, Inc. #5340",504-737-6242,9643-B JEFFERSON HWY,RIVER RIDGE,LA,https://www.cvs.com/store-locator/details-dire...,True,True,False,2024-08-01,29.957522,-90.217062
1,"CVS Pharmacy, Inc. #2113",215-836-1323,1600 WADSWORTH AVENUE,PHILADELPHIA,PA,https://www.cvs.com/store-locator/details-dire...,True,True,False,2024-08-01,40.080415,-75.171958
2,"CVS Pharmacy, Inc. #16504",619-315-0016,2911 JAMACHA RD,EL CAJON,CA,https://www.cvs.com/store-locator/details-dire...,True,True,True,2024-08-01,32.738217,-116.938327
3,Rite Aid #RA105255,253-851-6939,"4818 POINT FOSDICK DR.,NW",GIG HARBOR,WA,https://www.riteaid.com,True,False,False,2024-08-01,47.302477,-122.581456
4,Walmart Inc #10-1447,715-392-9520,3705 Tower Ave,Superior,WI,https://www.walmart.com/store/1447,True,True,False,2024-08-01,46.694518,-92.106958


In [18]:
dropvaccinetypes1_df = dropvaccinetypes_df.dropna()
print(dropvaccinetypes1_df)

                    Location Name  Phone Number                Street Address  \
0        CVS Pharmacy, Inc. #5340  504-737-6242          9643-B JEFFERSON HWY   
1        CVS Pharmacy, Inc. #2113  215-836-1323         1600 WADSWORTH AVENUE   
2       CVS Pharmacy, Inc. #16504  619-315-0016               2911 JAMACHA RD   
3              Rite Aid #RA105255  253-851-6939     4818 POINT FOSDICK DR.,NW   
4            Walmart Inc #10-1447  715-392-9520                3705 Tower Ave   
...                           ...           ...                           ...   
135780      SAV-ON PHARMACY #3739  215-942-4894               105 E STREET RD   
135781         Rite Aid #RA101653  603-532-6955        14 PETERBOROUGH STREET   
135782   CVS Pharmacy, Inc. #4378  540-989-6633           4400 BRAMBLETON AVE   
135783         Rite Aid #RA101963  724-438-9799      262 CONNELLSVILLE STREET   
135784        VONS PHARMACY #1962  951-695-7273  29530 RANCHO CALIFORNIA ROAD   

                        Cit

In [19]:
novactype_duplicates_df = dropvaccinetypes1_df.drop_duplicates()
print(novactype_duplicates_df)

                        Location Name  Phone Number  \
0            CVS Pharmacy, Inc. #5340  504-737-6242   
1            CVS Pharmacy, Inc. #2113  215-836-1323   
2           CVS Pharmacy, Inc. #16504  619-315-0016   
3                  Rite Aid #RA105255  253-851-6939   
4                Walmart Inc #10-1447  715-392-9520   
...                               ...           ...   
135748       CVS Pharmacy, Inc. #5773  718-424-2846   
135752  Publix Super Markets Inc. #17  803-407-0436   
135761       CVS Pharmacy, Inc. #3607  954-432-6804   
135765             Rite Aid #RA105326  503-698-8446   
135781             Rite Aid #RA101653  603-532-6955   

                   Street Address            City State  \
0            9643-B JEFFERSON HWY     RIVER RIDGE    LA   
1           1600 WADSWORTH AVENUE    PHILADELPHIA    PA   
2                 2911 JAMACHA RD        EL CAJON    CA   
3       4818 POINT FOSDICK DR.,NW      GIG HARBOR    WA   
4                  3705 Tower Ave        Sup

In [20]:
novactype_duplicates_df.head()

,Location Name,Phone Number,Street Address,City,State,Website,Insurance Accepted,Walk-Ins Accepted,In Stock,Last Updated,Latitude,Longitude
0,"CVS Pharmacy, Inc. #5340",504-737-6242,9643-B JEFFERSON HWY,RIVER RIDGE,LA,https://www.cvs.com/store-locator/details-dire...,True,True,False,2024-08-01,29.957522,-90.217062
1,"CVS Pharmacy, Inc. #2113",215-836-1323,1600 WADSWORTH AVENUE,PHILADELPHIA,PA,https://www.cvs.com/store-locator/details-dire...,True,True,False,2024-08-01,40.080415,-75.171958
2,"CVS Pharmacy, Inc. #16504",619-315-0016,2911 JAMACHA RD,EL CAJON,CA,https://www.cvs.com/store-locator/details-dire...,True,True,True,2024-08-01,32.738217,-116.938327
3,Rite Aid #RA105255,253-851-6939,"4818 POINT FOSDICK DR.,NW",GIG HARBOR,WA,https://www.riteaid.com,True,False,False,2024-08-01,47.302477,-122.581456
4,Walmart Inc #10-1447,715-392-9520,3705 Tower Ave,Superior,WI,https://www.walmart.com/store/1447,True,True,False,2024-08-01,46.694518,-92.106958


In [21]:
novactypestock_df = novactype_duplicates_df.drop(columns=['In Stock'])
print(novactypestock_df)

                        Location Name  Phone Number  \
0            CVS Pharmacy, Inc. #5340  504-737-6242   
1            CVS Pharmacy, Inc. #2113  215-836-1323   
2           CVS Pharmacy, Inc. #16504  619-315-0016   
3                  Rite Aid #RA105255  253-851-6939   
4                Walmart Inc #10-1447  715-392-9520   
...                               ...           ...   
135748       CVS Pharmacy, Inc. #5773  718-424-2846   
135752  Publix Super Markets Inc. #17  803-407-0436   
135761       CVS Pharmacy, Inc. #3607  954-432-6804   
135765             Rite Aid #RA105326  503-698-8446   
135781             Rite Aid #RA101653  603-532-6955   

                   Street Address            City State  \
0            9643-B JEFFERSON HWY     RIVER RIDGE    LA   
1           1600 WADSWORTH AVENUE    PHILADELPHIA    PA   
2                 2911 JAMACHA RD        EL CAJON    CA   
3       4818 POINT FOSDICK DR.,NW      GIG HARBOR    WA   
4                  3705 Tower Ave        Sup

In [22]:
#remove in stock duplicates
novactypestockduplicate_df = novactypestock_df.drop_duplicates()
print(novactypestockduplicate_df)

                    Location Name  Phone Number             Street Address  \
0        CVS Pharmacy, Inc. #5340  504-737-6242       9643-B JEFFERSON HWY   
1        CVS Pharmacy, Inc. #2113  215-836-1323      1600 WADSWORTH AVENUE   
2       CVS Pharmacy, Inc. #16504  619-315-0016            2911 JAMACHA RD   
3              Rite Aid #RA105255  253-851-6939  4818 POINT FOSDICK DR.,NW   
4            Walmart Inc #10-1447  715-392-9520             3705 Tower Ave   
...                           ...           ...                        ...   
135309         Rite Aid #RA104695  586-463-8661     28350 SOUTH RIVER ROAD   
135452         Rite Aid #RA107884  717-245-0116              320 YORK ROAD   
135536         Rite Aid #RA105513  310-831-9167    501 SOUTH GAFFEY STREET   
135765         Rite Aid #RA105326  503-698-8446    12002 SE SUNNYSIDE ROAD   
135781         Rite Aid #RA101653  603-532-6955     14 PETERBOROUGH STREET   

                     City State  \
0             RIVER RIDGE   

In [23]:
#Save DataFrame to a csv
novactypestockduplicate_df.to_csv("novactypestock.csv")
# This is the final CSV file as of Nov 25, 2024, ~ 9 pm PST.

In [25]:
novactypestockduplicate_df.columns

Index(['Location Name', 'Phone Number', 'Street Address', 'City', 'State',
       'Website', 'Insurance Accepted', 'Walk-Ins Accepted', 'Last Updated',
       'Latitude', 'Longitude'],
      dtype='object')

In [ ]:
def get_zipcode_from_coordinates(latitude, longitude):

    api_key = "YOUR_GOOGLE_MAPS_API_KEY"  # Replace with your actual API key

    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={api_key}"

    response = requests.get(url)

    data = response.json()

    

    if data['status'] == 'OK':

        zip_code = data['results'][0]['address_components'][0]['short_name']  # Access the zip code from the response

        return zip_code

    else:

        return None


In [28]:

# Initialize geolocator
geolocator = Nominatim(user_agent="my_geocoder")

def get_zipcode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        return location.raw['address'].get('postcode', None)
    except GeocoderTimedOut:
        return get_zipcode(lat, lon)  # Retry on timeout
    except Exception as e:
        return None


# Apply the function to each row
df['Zipcode'] = novactypestockduplicate_df.apply(lambda row: get_zipcode(row['Latitude'], row['Longitude']), axis=1)
print(df)

NameError: name 'GeocoderTimedOut' is not defined

In [ ]:
novactypestockduplicate_df.columns

## Storing of Data in MongoDB, needs to be updated by Rowan

In the terminal, import data from csv using the following code: 
mongoImport --type csv -d vaccines_db -c vaccine_providers --headerline --drop Vaccine Providers.csv

In [ ]:
# Module used to connect Python with MongoDB
from pymongo import MongoClient

In [ ]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
mongo = MongoClient(port=27017)

# Define the 'vaccines_db' database in Mongo
db = mongo.vaccines_db

# Declare the collection
providers = db.vaccine_providers

In [ ]:
#Check that the data has been stored by making a query from the collection
location = providers.find()

print(location[0])